In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

In [2]:
Link = 'https://www.fangraphs.com/' + 'statsd.aspx?playerid=14109&position=OF&type=1&gds=&gde=&season=all'

In [3]:
Link

'https://www.fangraphs.com/statsd.aspx?playerid=14109&position=OF&type=1&gds=&gde=&season=all'

In [4]:
rStd = requests.get(Link)
soupStd = BeautifulSoup(rStd.text, "html.parser")
rAvd = requests.get(Link.replace("type=1", "type=2"))
soupAvd = BeautifulSoup(rAvd.text, "html.parser")

In [5]:
#Get Stats tables - Standard
StdTable = soupStd.find_all("tr", "rgRow")
StdTableAlt = soupStd.find_all("tr", "rgAltRow")

#Get Total Stats - Advanced
AvdTable = soupAvd.find_all("tr", "rgRow")
AvdTableAlt = soupAvd.find_all("tr", "rgAltRow")

In [6]:
GameLog = []
n = len(StdTable)
#n=2
for i in range(n):
    #i == 0 is the total line.
    row = []
    gridrow = StdTable[i].find_all("td", "grid_line_regular")
    for j in range(len(gridrow)):
        if j < 5 :
            row.append(gridrow[j].text)
        else:
            row.append(float(gridrow[j].text))
    GameLog.append(row)
n = len(StdTableAlt)
for i in range(n):
    #i == 0 is the total line.
    row = []
    gridrow = StdTableAlt[i].find_all("td", "grid_line_regular")
    for j in range(len(gridrow)):
        if j < 5 :
            row.append(gridrow[j].text)
        else:
            row.append(float(gridrow[j].text))
    GameLog.append(row)

In [7]:
GameLogSTDDF = pd.DataFrame(data=GameLog, columns=['Date', 'Team','Opp','BO','Pos','AB','PA','H','1B', '2B', '3B','HR','R','RBI','BB','IBB','SO', 'HBP', 'SF', 'SH', 'GDP', 'SB', 'CS', 'AVG'])

In [8]:
GameLogSTDDF.head()

,Date,Team,Opp,BO,Pos,AB,PA,H,1B,2B,...,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG
0,Total,- - -,- - -,- - -,- - -,672.0,723.0,205.0,141.0,46.0,...,41.0,2.0,118.0,1.0,5.0,4.0,18.0,2.0,2.0,0.305
1,2018-07-08,CHC,CIN,1,CF,5.0,5.0,2.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.400
2,2018-07-06,CHC,CIN,1,CF,4.0,4.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250
3,2018-07-03,CHC,DET,1,CF,4.0,5.0,1.0,0.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250
4,2018-06-30,CHC,MIN,1,CF,3.0,4.0,3.0,3.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.000


In [9]:
rows, col = GameLogSTDDF.shape
rows

262

In [10]:
#Calculate DK fps
GameLogSTDDF['DKfp'] = 0.0
Sum = 0
for i in range(1,rows):
    GameLogSTDDF['DKfp'][i] = 3*GameLogSTDDF['1B'][i] + 5*GameLogSTDDF['2B'][i] + 8*GameLogSTDDF['3B'][i] + 10*GameLogSTDDF['HR'][i] + 2*GameLogSTDDF['RBI'][i]+2*GameLogSTDDF['R'][i]+2*GameLogSTDDF['BB'][i]+2*GameLogSTDDF['HBP'][i]+5*GameLogSTDDF['SB'][i]
    Sum = Sum + GameLogSTDDF['DKfp'][i]
GameLogSTDDF['DKfp'][0] = Sum

/home/josh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/josh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [11]:
GameLogSTDDF.head()

,Date,Team,Opp,BO,Pos,AB,PA,H,1B,2B,...,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,DKfp
0,Total,- - -,- - -,- - -,- - -,672.0,723.0,205.0,141.0,46.0,...,2.0,118.0,1.0,5.0,4.0,18.0,2.0,2.0,0.305,1297.0
1,2018-07-08,CHC,CIN,1,CF,5.0,5.0,2.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.400,8.0
2,2018-07-06,CHC,CIN,1,CF,4.0,4.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250,5.0
3,2018-07-03,CHC,DET,1,CF,4.0,5.0,1.0,0.0,1.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250,11.0
4,2018-06-30,CHC,MIN,1,CF,3.0,4.0,3.0,3.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.000,15.0


In [12]:
#Get Pitching stats.
Link = 'https://www.fangraphs.com/' + 'statsd.aspx?playerid=4930&position=P&type=1&gds=&gde=&season=all'

In [13]:
Link

'https://www.fangraphs.com/statsd.aspx?playerid=4930&position=P&type=1&gds=&gde=&season=all'

In [14]:
rStd = requests.get(Link)
soupStd = BeautifulSoup(rStd.text, "html.parser")

In [15]:
StdTable = soupStd.find_all("tr", "rgRow")
StdTableAlt = soupStd.find_all("tr", "rgAltRow")

In [16]:
GameLog = []
n = len(StdTable)

for i in range(n):
    #i == 0 is the total line.
    row = []
    gridrow = StdTable[i].find_all("td")
    for j in range(len(gridrow)):
        if j < 3 :
            row.append(gridrow[j].text)
        else:
            try:
                row.append(float(gridrow[j].text))
            except:
                row.append(gridrow[j].text)
    GameLog.append(row)
n = len(StdTableAlt)
for i in range(n):
    #i == 0 is the total line.
    row = []
    gridrow = StdTableAlt[i].find_all("td")
    for j in range(len(gridrow)):
        if j < 3 :
            row.append(gridrow[j].text)
        else:
            try:
                row.append(float(gridrow[j].text))
            except:
                row.append(gridrow[j].text)
                
    GameLog.append(row)

In [17]:
GameLogPDPit=pd.DataFrame(data=GameLog, columns=['Date','Team','Opp','GS','W','L','ERA','G','GS','CG','ShO','SV','HLD','BS','IP','TBF','H','R','ER','HR','BB','IBB','HBP','WP','BK','SO','GSv2'])

In [18]:
GameLogPDPit.head()

,Date,Team,Opp,GS,W,L,ERA,G,GS,CG,...,R,ER,HR,BB,IBB,HBP,WP,BK,SO,GSv2
0,Total,- - -,- - -,366.0,170.0,94.0,3.47,367.0,366.0,15.0,...,955.0,882.0,220.0,742.0,8.0,78.0,56.0,1.0,2122.0,56
1,2018-07-08,CHC,CIN,1.0,0.0,0.0,5.68,1.0,1.0,0.0,...,4.0,4.0,1.0,3.0,0.0,0.0,0.0,0.0,2.0,43
2,2018-07-01,CHC,MIN,1.0,1.0,0.0,3.60,1.0,1.0,0.0,...,4.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0,5.0,32
3,2018-06-20,CHC,LAD,1.0,1.0,0.0,0.00,1.0,1.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,66
4,2018-06-09,CHC,PIT,1.0,1.0,0.0,0.00,1.0,1.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,5.0,80


In [20]:
#Calculate DK fps
rows, cols = GameLogPDPit.shape
Sum = 0
GameLogPDPit['DKfp'] = 2.25*GameLogPDPit['IP'] + 2*GameLogPDPit['SO'] + 4*GameLogPDPit['W'] - 2*GameLogPDPit['ER']- 0.6*GameLogPDPit['BB'] - 0.6*GameLogPDPit['HBP']
for i in range(1,rows):
    if GameLogPDPit['IP'][i] == 9:
        GameLogPDPit['DKfp'][i] = GameLogPDPit['DKfp'][i] + 2.5
        if GameLogPDPit['ER'][i] == 0:
            GameLogPDPit['DKfp'][i] = GameLogPDPit['DKfp'][i] + 2.5
    if GameLogPDPit['H'][i] == 0:
        GameLogPDPit['DKfp'][i] = GameLogPDPit['DKfp'][i] + 5       
    GameLogPDPit['DKfp'][i] = FP
    Sum = Sum + FP
GameLogPDPit['DKfp'][0] = Sum

,Date,Team,Opp,GS,W,L,ERA,G,GS,CG,...,ER,HR,BB,IBB,HBP,WP,BK,SO,GSv2,DKfp
0,Total,- - -,- - -,366.0,170.0,94.0,3.47,367.0,366.0,15.0,...,882.0,220.0,742.0,8.0,78.0,56.0,1.0,2122.0,56,7820.950
1,2018-07-08,CHC,CIN,1.0,0.0,0.0,5.68,1.0,1.0,0.0,...,4.0,1.0,3.0,0.0,0.0,0.0,0.0,2.0,43,7.925
2,2018-07-01,CHC,MIN,1.0,1.0,0.0,3.60,1.0,1.0,0.0,...,2.0,1.0,3.0,0.0,0.0,0.0,0.0,5.0,32,19.450
3,2018-06-20,CHC,LAD,1.0,1.0,0.0,0.00,1.0,1.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,66,19.950
4,2018-06-09,CHC,PIT,1.0,1.0,0.0,0.00,1.0,1.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,5.0,80,28.550
5,2018-05-29,CHC,@PIT,1.0,1.0,0.0,6.00,1.0,1.0,0.0,...,4.0,2.0,0.0,0.0,1.0,0.0,0.0,6.0,41,20.900
6,2018-05-18,CHC,@CIN,1.0,1.0,0.0,1.50,1.0,1.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,8.0,68,30.900
7,2018-05-06,CHC,@STL,1.0,0.0,0.0,3.38,1.0,1.0,0.0,...,2.0,1.0,4.0,1.0,0.0,0.0,0.0,4.0,43,13.075
8,2018-04-25,CHC,@CLE,1.0,0.0,1.0,3.86,1.0,1.0,0.0,...,3.0,3.0,2.0,0.0,0.0,0.0,0.0,4.0,46,16.550
9,2018-04-11,CHC,PIT,1.0,0.0,0.0,7.20,1.0,1.0,0.0,...,4.0,1.0,3.0,0.0,0.0,0.0,0.0,3.0,34,7.450
